In [11]:
import boto3
import pandas as pd
from io import StringIO, BytesIO
from datetime import datetime, timedelta

In [12]:
class Extract():
    def __init__(self, arg_date, src_format, src_bucket, trg_bucket, columns, key, s3, bucket, objects, df_all, bucket_target):
        
        # Parameters/Configurations
        self.arg_date = arg_date
        self.src_format = src_format
        self.src_bucket = src_bucket
        self.trg_bucket = trg_bucket
        self.columns = columns
        self.key = key
        self.bucket = bucket
        self.objects = objects
        self.df_all = df_all
        self.bucket_target = bucket_target

        arg_date = '2022-03-02'
        src_format = '%Y-%m-%d'
        src_bucket = 'deutsche-boerse-xetra-pds'
        trg_bucket = 'xetra-bucket-2022-03-02-gerardo'
        columns = ['ISIN', 'Date', 'Time', 'StartPrice', 'MaxPrice', 'MinPrice', 'EndPrice', 'TradedVolume']
        key = 'xetra_daily_report_' + datetime.today().strftime("%Y%m%d_%H%M%S") + '.parquet'
        
        s3 = boto3.resource('s3')
        bucket = s3.Bucket(src_bucket)
        objects = return_objects(src_bucket, src_format, arg_date, bucket)
        df_all = extract(objects, bucket, columns, key)
        df_all = transform_report(df_all, arg_date, columns)
        bucket_target = write_df_to_s3(trg_bucket, key, df_all, s3)
        

def read_csv_to_df(self, filename):
    csv_obj = self.bucket.Object(key=filename).get().get('Body').read().decode('utf-8')
    data = StringIO(csv_obj)
    df = pd.read_csv(data, delimiter=',')
    return df

def write_df_to_s3(self):
    out_buffer = BytesIO()
    self.df_all.to_parquet(out_buffer, index=False)
    bucket_target = self.s3.Bucket(self.trg_bucket)
    bucket_target.put_object(Body=out_buffer.getvalue(), Key=self.key)
    return bucket_target

def return_objects(self):
    arg_date_dt = datetime.strptime(self.arg_date, self.src_format).date() - timedelta(days=1)
    objects = [obj for obj in self.bucket.objects.all() if datetime.strptime(obj.key.split('/')[0], self.src_format).date() >= arg_date_dt]
    ##objKey= obj.key
    return objects

def extract(self):
    df_all = pd.concat([read_csv_to_df(obj.self.key, self.bucket) for obj in self.objects], ignore_index=True)
    df_all = df_all.loc[:, self.columns]
    df_all.dropna(inplace=True)
    return df_all

In [13]:
class Tranform(Extract):
    def __init__(self, arg_date, columns, df_all):
        super(Tranform, self).__init__(self, arg_date, columns, df_all)

def transform_report(self, df_all, arg_date):
    self.df_all['opening_price'] = df_all.sort_values(by=['Time']).groupby(['ISIN', 'Date'])['StartPrice'].transform('first')
    self.df_all['closing_price'] = df_all.sort_values(by=['Time']).groupby(['ISIN', 'Date'])['StartPrice'].transform('last')
    self.df_all = df_all.groupby(['ISIN', 'Date'], as_index=False).agg(opening_price_eur=('opening_price', 'min'), closing_price_eur=('closing_price', 'min'), minimum_price_eur=('MinPrice', 'min'), maximum_price_eur=('MaxPrice', 'max'), daily_traded_volume=('TradedVolume', 'sum'))
    self.df_all['prev_closing_price'] = df_all.sort_values(by=['Date']).groupby(['ISIN'])['closing_price_eur'].shift(1)
    self.df_all['change_prev_closing_%'] = (df_all['closing_price_eur'] - df_all['prev_closing_price']) / df_all['prev_closing_price'] * 100
    self.df_all.drop(columns=['prev_closing_price'], inplace=True)
    self.df_all = df_all.round(decimals=2)
    self.df_all = df_all[df_all.Date >= arg_date]
    return df_all
        

In [14]:
class Load(Tranform):
    def __init__(self, bucket_target, key, s3):
        super(Tranform, self).__init__(self, bucket_target, key, s3)

def load(self, bucket_target):

    objKey=[]
    for obj in self.bucket_target.objects.all():
        objKey.append(obj.key)
    prq_obj = bucket_target.Object(key=objKey[-1]).get().get('Body').read()
    data = BytesIO(prq_obj)
    return data

def etl_report(self):
    df_report = pd.read_parquet(load(self.bucket_target, self.key, selfs3))
    return df_report

In [16]:
class Main():
    def main(self):
        print(etl_report(self.bucket_target, self.key, self.s3))

In [2]:
Main()
Extract()
Tranform()
Load()

NameError: name 'Main' is not defined